In [1]:
import mitoolspro as mtp
import pandas as pd
import numpy as np
from mitoolspro.project import Project
from mitoolspro import economic_complexity as ec

In [ ]:
pr = Project.load(auto_load=True)

# DataBases

In [3]:
raw_db = mtp.databases.MainConnection(pr.get_path('raw_db_path'))
db = mtp.databases.MainConnection(pr.get_path("database"))

# Parameters

In [4]:
years = pr.vars['years']
raw_tablenames = [f'oec_hs_trade_{year}' for year in years]

In [5]:
time_id = pr.vars['time_id']
data_id = pr.vars['data_id']
raw_tablename = ec.create_data_name(data_id, 'raw')
clean_tablename = ec.create_data_name(data_id, 'clean')

In [6]:
countries_population_tablename = 'country_population'

#### HS Codes

In [7]:
hs_codes = pd.read_sql("SELECT * FROM hs_codes", raw_db, index_col='index')

In [8]:
hs_codes = hs_codes[['Section', 'Section ID', 'HS2', 'HS2 ID', 'HS4', 'HS4 ID', 'HS6', 'HS6 ID']].drop_duplicates()
hs6 = hs_codes[['HS6 ID', 'HS6']].drop_duplicates()
hs_codes = hs_codes.loc[~hs6['HS6'].duplicated(keep='last')].reset_index(drop=True).sort_values(by=['Section ID', 'HS2 ID', 'HS4 ID', 'HS6 ID'])

In [ ]:
hs_codes.to_sql('hs_codes', db, if_exists='replace')

In [10]:
create_clean_table = not mtp.databases.check_if_table(db, clean_tablename)

# Transfer Raw Data

In [11]:
if not mtp.databases.check_if_table(db, raw_tablename):
    data = pd.concat(mtp.databases.read_sql_tables(raw_db, raw_tablenames), axis=0).reset_index(drop=True)
    data = data.astype({
        'Year': np.int32,
        'HS2 ID': np.int32,
        'HS4 ID': np.int32,
        'HS6 ID': np.int32,
        'Trade Value': np.float32
    })
    data.to_sql(raw_tablename, db, if_exists='replace', index=False)
else:
    data = pd.read_sql(f"SELECT * FROM {raw_tablename}", db)

# Standardize Countries Names

In [12]:
cc = ec.name_converter

#### Remove Inexistent Countries

In [13]:
data = data.loc[~data['Country'].isin(['Belgium-Luxembourg', 'Yugoslavia'])].copy(deep=True)

#### Map Names

In [14]:
countries = data['Country'].unique()
standardized_countries = cc.convert(countries, to='name_short')
countries_map = {c: s for c, s in zip(countries, standardized_countries)}

In [15]:
standardized_id_countries = cc.convert(standardized_countries, to='ISO3')
id_countries_map = {c: s for c, s in zip(standardized_countries, standardized_id_countries)}

In [16]:
data['Country'] = data['Country'].map(countries_map)
data['Country ID'] = data['Country'].map(id_countries_map)

# Filter by Country Population

In [17]:
if not mtp.databases.check_if_table(db, countries_population_tablename):
    countries_population = pd.read_sql(f'SELECT * FROM {countries_population_tablename}', raw_db, index_col='index')
    countries_population.to_sql(countries_population_tablename, db, if_exists='replace')
else:
    countries_population = pd.read_sql(f'SELECT * FROM {countries_population_tablename}', db, index_col='index')

In [18]:
countries_population = countries_population.loc[~countries_population['Country'].isin(['Channel Islands', 'Virgin Islands', 'Caribbean', 'World'])]
countries_population.columns = countries_population.columns.map({
    c: c.replace(' ', '') for c in countries_population.columns
})

In [19]:
pop_countries = countries_population['Country'].unique()
standardized_pop_countries = cc.convert(pop_countries, to='name_short')
pop_countries_map = {c: s for c, s in zip(pop_countries, standardized_pop_countries)}

In [20]:
standardized_id_countries = cc.convert(pop_countries, to='ISO3')
id_countries_map = {c: s for c, s in zip(pop_countries, standardized_id_countries)}

In [21]:
countries_population['Country'] = countries_population['Country'].map(pop_countries_map)
countries_population['Country ID'] = countries_population['Country'].map(id_countries_map)

- Filtering countries that have less than 1.000.000 inhabitants by 2020.

In [22]:
yearly_pop = countries_population.loc[countries_population['Year'] == 2020]
populated_countries = yearly_pop.loc[yearly_pop['Measure'] >= 1_000_000]

Apply filter:

In [23]:
data = data.loc[data['Country'].isin(populated_countries['Country'].values)]

# Filter by Amount of Exports

In [24]:
countries_exports = data[['Year', 'Country', 'Trade Value']].groupby(by=['Year', 'Country']).sum()[['Trade Value']].reset_index()
exporting_countries = countries_exports.loc[countries_exports['Trade Value'] >= 1_000_000_000, 'Country'].unique()

Apply filter:

In [25]:
data = data.loc[data['Country'].isin(exporting_countries)]

# Save Clean Data

In [ ]:
data = data.groupby([c for c in data.columns if c != 'Trade Value']).sum().reset_index()
data.to_sql(clean_tablename, db, if_exists='replace', index=False)

***